In [ ]:
#以下代码为基于人眼位置的人脸对齐

In [ ]:
#根据眼睛坐标对图像进行仿射变换
#src - 原图像
#landmarks - 原图像中68个关键点
Mat getwarpAffineImg(Mat &src, vector<Point2f> &landmarks)
{
    Mat oral;src.copyTo(oral);
    for (int j = 0; j < landmarks.size(); j++)
    {
        circle(oral, landmarks[j], 2, Scalar(255, 0, 0));
    }
    #计算两眼中心点，按照此中心点进行旋转， 第31个为左眼坐标，36为右眼坐标
    Point2f eyesCenter = Point2f( (landmarks[31].x + landmarks[36].x) * 0.5f, (landmarks[31].y + landmarks[36].y) * 0.5f );
    
    # 计算两个眼睛间的角度
    double dy = (landmarks[36].y - landmarks[31].y);
    double dx = (landmarks[36].x - landmarks[31].x);
    double angle = atan2(dy, dx) * 180.0/CV_PI; // Convert from radians to degrees.
    
    #由eyesCenter, andle, scale按照公式计算仿射变换矩阵，此时1.0表示不进行缩放
    Mat rot_mat = getRotationMatrix2D(eyesCenter, angle, 1.0);
    Mat rot;
    # 进行仿射变换，变换后大小为src的大小
    warpAffine(src, rot, rot_mat, src.size());
    vector<Point2f> marks;
    
    #按照仿射变换矩阵，计算变换后各关键点在新图中所对应的位置坐标。
    for (int n = 0; n<landmarks.size(); n++)
    {
        Point2f p = Point2f(0, 0);
        p.x = rot_mat.ptr<double>(0)[0] *landmarks[n].x + rot_mat.ptr<double>(0)[1] * landmarks[n].y + rot_mat.ptr<double>(0)[2];
        p.y = rot_mat.ptr<double>(1)[0] * landmarks[n].x + rot_mat.ptr<double>(1)[1] * landmarks[n].y + rot_mat.ptr<double>(1)[2];
        marks.push_back(p);
    }
    #标出关键点
    for (int j = 0; j < landmarks.size(); j++)
    {
        circle(rot, marks[j], 2, Scalar(0, 0, 255));
    }
    return rot;
}

In [ ]:
# -*- coding: utf-8 -*-  
# 参数含义：  
#  CropFace(image, eye_left, eye_right, offset_pct, dest_sz)  
# eye_left is the position of the left eye  
# eye_right is the position of the right eye  
# 比例的含义为：要保留的图像靠近眼镜的百分比，  
# offset_pct is the percent of the image you want to keep next to the eyes (horizontal, vertical direction)  
# 最后保留的图像的大小。  
# dest_sz is the size of the output image  
#  
import sys,math,Image  
   
 # 计算两个坐标的距离  
def Distance(p1,p2):
    dx = p2[0]- p1[0]  
    dy = p2[1]- p1[1]  
    return math.sqrt(dx*dx+dy*dy)  
   
 # 根据参数，求仿射变换矩阵和变换后的图像。  
def ScaleRotateTranslate(image, angle, center =None, new_center =None, scale =None, resample=Image.BICUBIC):  
    if (scale is None)and (center is None):  
        return image.rotate(angle=angle, resample=resample)  
    nx,ny = x,y = center  
    sx=sy=1.0  
    if new_center:  
        (nx,ny) = new_center  
    if scale:  
        (sx,sy) = (scale, scale)  
    cosine = math.cos(angle)  
    sine = math.sin(angle)  
    a = cosine/sx  
    b = sine/sx  
    c = x-nx*a-ny*b  
    d =-sine/sy  
    e = cosine/sy  
    f = y-nx*d-ny*e  
    return image.transform(image.size, Image.AFFINE, (a,b,c,d,e,f), resample=resample)

 # 根据所给的人脸图像，眼睛坐标位置，偏移比例，输出的大小，来进行裁剪。  
def CropFace(image, eye_left=(0,0), eye_right=(0,0), offset_pct=(0.2,0.2), dest_sz = (70,70)):
    # calculate offsets in original image 计算在原始图像上的偏移。 
    offset_h = math.floor(float(offset_pct[0])*dest_sz[0])  
    offset_v = math.floor(float(offset_pct[1])*dest_sz[1])  
    # get the direction  计算眼睛的方向。  
    eye_direction = (eye_right[0]- eye_left[0], eye_right[1]- eye_left[1])  
    # calc rotation angle in radians  计算旋转的方向弧度。  
    rotation =-math.atan2(float(eye_direction[1]),float(eye_direction[0]))  
    # distance between them  # 计算两眼之间的距离。  
    dist = Distance(eye_left, eye_right)  
    # calculate the reference eye-width    计算最后输出的图像两只眼睛之间的距离。  
    reference = dest_sz[0]-2.0*offset_h  
    # scale factor   # 计算尺度因子。  
    scale =float(dist)/float(reference)  
    # rotate original around the left eye  # 原图像绕着左眼的坐标旋转。  
    image = ScaleRotateTranslate(image, center=eye_left, angle=rotation)  
    # crop the rotated image  # 剪切  
    crop_xy = (eye_left[0]- scale*offset_h, eye_left[1]- scale*offset_v)  # 起点  
    crop_size = (dest_sz[0]*scale, dest_sz[1]*scale)   # 大小  
    image = image.crop((int(crop_xy[0]),int(crop_xy[1]),int(crop_xy[0]+crop_size[0]),int(crop_xy[1]+crop_size[1])))  
    # resize it 重置大小  
    image = image.resize(dest_sz, Image.ANTIALIAS)  
    return image  
    
if __name__ =="__main__":
    image =  Image.open("/media/crw/DataCenter/Dataset/CAS-PEAL-R1/POSE/000001/MY_000001_IEU+00_PD-22_EN_A0_D0_T0_BB_M0_R1_S0.tif")  
    leftx =117  
    lefty=287  
    rightx=187  
    righty= 288  
    CropFace(image, eye_left=(leftx,lefty), eye_right=(rightx,righty), offset_pct=(0.1,0.1), dest_sz=(200,200)).save("test_10_10_200_200.jpg")  
    CropFace(image, eye_left=(leftx,lefty), eye_right=(rightx,righty), offset_pct=(0.2,0.2), dest_sz=(200,200)).save("test_20_20_200_200.jpg")  
    CropFace(image, eye_left=(leftx,lefty), eye_right=(rightx,righty), offset_pct=(0.3,0.3), dest_sz=(200,200)).save("test_30_30_200_200.jpg")  
    CropFace(image, eye_left=(leftx,lefty), eye_right=(rightx,righty), offset_pct=(0.4,0.4), dest_sz=(200,200)).save("test_40_40_200_200.jpg")  
    CropFace(image, eye_left=(leftx,lefty), eye_right=(rightx,righty), offset_pct=(0.45,0.45), dest_sz=(200,200)).save("test_45_45_200_200.jpg")  
    CropFace(image, eye_left=(leftx,lefty), eye_right=(rightx,righty), offset_pct=(0.2,0.2)).save("test_20_20_70_70.jpg")  